In [13]:
# Check support version for PyG
!pip list -v | grep torch

torch                         2.0.0+cu118          /usr/local/lib/python3.9/dist-packages pip
torch-cluster                 1.6.1+pt20cu118      /usr/local/lib/python3.9/dist-packages pip
torch-geometric               2.3.0                /usr/local/lib/python3.9/dist-packages pip
torch-scatter                 2.1.1+pt20cu118      /usr/local/lib/python3.9/dist-packages pip
torch-sparse                  0.6.17+pt20cu118     /usr/local/lib/python3.9/dist-packages pip
torch-spline-conv             1.2.2+pt20cu118      /usr/local/lib/python3.9/dist-packages pip
torchaudio                    2.0.1+cu118          /usr/local/lib/python3.9/dist-packages pip
torchdata                     0.6.0                /usr/local/lib/python3.9/dist-packages pip
torchsummary                  1.5.1                /usr/local/lib/python3.9/dist-packages pip
torchtext                     0.15.1               /usr/local/lib/python3.9/dist-packages pip
torchvision                   0.15.1+cu118         /usr/loca

In [14]:
# Install the PyG library
!pip install torch_geometric
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html


In [16]:
# Import statements
import torch
import torch_geometric
import torch.nn.functional as F
torch.manual_seed(548)

In [18]:
# Load the dataset
planetoid = torch_geometric.datasets.Planetoid(root='/data/CiteSeer', name='CiteSeer')
data = planetoid[0]

Processing...
Done!


In [19]:
# Print and check the contents
data
data.y

tensor([3, 1, 5,  ..., 3, 1, 5])

In [20]:
# Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, dataset):
        super().__init__()
        self.conv1 = torch_geometric.nn.GCNConv(dataset.num_features, 16)
        self.conv2 = torch_geometric.nn.GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        return x

In [21]:
# Initialize model and set hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
data = data.to(device)
model = GCN(planetoid).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

cpu


In [25]:
from torch.cuda import default_stream
# Train model
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
      pred = out.argmax(dim=1)
      train_acc = pred[data.train_mask].eq(data.y[data.train_mask]).sum().item() / data.train_mask.sum().item()
      val_acc = pred[data.val_mask].eq(data.y[data.val_mask]).sum().item() / data.val_mask.sum().item()
      print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

Epoch: 0, Loss: 0.0072, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 10, Loss: 0.0070, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 20, Loss: 0.0069, Train Acc: 1.0000, Val Acc: 0.6880
Epoch: 30, Loss: 0.0067, Train Acc: 1.0000, Val Acc: 0.6900
Epoch: 40, Loss: 0.0066, Train Acc: 1.0000, Val Acc: 0.6900
Epoch: 50, Loss: 0.0064, Train Acc: 1.0000, Val Acc: 0.6880
Epoch: 60, Loss: 0.0063, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 70, Loss: 0.0062, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 80, Loss: 0.0061, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 90, Loss: 0.0060, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 100, Loss: 0.0059, Train Acc: 1.0000, Val Acc: 0.6840
Epoch: 110, Loss: 0.0059, Train Acc: 1.0000, Val Acc: 0.6840
Epoch: 120, Loss: 0.0058, Train Acc: 1.0000, Val Acc: 0.6840
Epoch: 130, Loss: 0.0057, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 140, Loss: 0.0057, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 150, Loss: 0.0056, Train Acc: 1.0000, Val Acc: 0.6860
Epoch: 160, Loss: 0.0056, Train Acc

Next, we evaluate the model on the testing side.

In [26]:
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.6840
